<a href="https://colab.research.google.com/github/ganeshiimi/AV/blob/master/NLPProject_I_Social_Media_Information_Extraction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
#nltk.download('wordnet')
#nltk.download('stopwords')
nltk.download("popular")
from nltk.tokenize import sent_tokenize,word_tokenize
from textblob import TextBlob

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [ ]:
file=r'/content/drive/My Drive/NLP/tweets.csv'
#Load the dataset

dataset = pd.read_csv(file, encoding = "ISO-8859-1")

dataset.head()


,Unnamed: 0,X,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted
0,1,1,RT @rssurjewala: Critical question: Was PayTM ...,False,0,NaN,2016-11-23 18:40:30,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",HASHTAGFARZIWAL,331,True,False
1,2,2,RT @Hemant_80: Did you vote on #Demonetization...,False,0,NaN,2016-11-23 18:40:29,False,NaN,8.014957e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",PRAMODKAUSHIK9,66,True,False
2,3,3,"RT @roshankar: Former FinSec, RBI Dy Governor,...",False,0,NaN,2016-11-23 18:40:03,False,NaN,8.014955e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",rahulja13034944,12,True,False
3,4,4,RT @ANI_news: Gurugram (Haryana): Post office ...,False,0,NaN,2016-11-23 18:39:59,False,NaN,8.014955e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",deeptiyvd,338,True,False
4,5,5,RT @satishacharya: Reddy Wedding! @mail_today ...,False,0,NaN,2016-11-23 18:39:39,False,NaN,8.014954e+17,NaN,"<a href=""http://cpimharyana.com"" rel=""nofollow...",CPIMBadli,120,True,False


In [ ]:
for index, tweet in enumerate(dataset["text"][10:15]):
    print(index+1,".",tweet)

1 . Many opposition leaders are with @narendramodi on the #Demonetization 
And respect their decision,but support opposition just b'coz of party
2 . RT @Joydas: Question in Narendra Modi App where PM is taking feedback if people support his #DeMonetization strategy https://t.co/pYgK8Rmg7r
3 . @Jaggesh2 Bharat band on 28??<ed><U+00A0><U+00BD><ed><U+00B8><U+0082>Those who  are protesting #demonetization  are all different party leaders.
4 . RT @Atheist_Krishna: The effect of #Demonetization !!
. https://t.co/A8of7zh2f5
5 . RT @sona2905: When I explained #Demonetization to myself and tried to put it down in my words which are not laced with any heavy technical


In [ ]:
#Preprocessing
punc=string.punctuation
punc=punc.replace('@', '')
stop=stopwords.words("english")
lemmatizer = WordNetLemmatizer() 
def _clean(text):
  text=text.lower()
  text="".join(x for x in text if x not in punc) 
  words=text.split()
  words=[w for w in words if w not in stop]
  text=" ".join(words) 
  text=" ".join(w for w in text.split(' ') if not w.startswith('rt @'))
  text=text.replace("rt "," ")
  text=re.sub(r'@\w+', '', text)
  text=lemmatizer.lemmatize(text)
  return text

In [ ]:
dataset['cleaned']=dataset['text'].apply(_clean)
dataset[['text','cleaned']].head()

,text,cleaned
0,RT @rssurjewala: Critical question: Was PayTM ...,critical question paytm informed demonetizat...
1,RT @Hemant_80: Did you vote on #Demonetization...,vote demonetization modi survey app
2,"RT @roshankar: Former FinSec, RBI Dy Governor,...",former finsec rbi dy governor cbdt chair har...
3,RT @ANI_news: Gurugram (Haryana): Post office ...,gurugram haryana post office employees provi...
4,RT @satishacharya: Reddy Wedding! @mail_today ...,reddy wedding cartoon demonetization reddyw...


In [ ]:
complete_text=" ".join(dataset['cleaned'])
words=complete_text.split()
Counter(words).most_common(7)

[('demonetization', 13939),
 ('india', 2766),
 ('modi', 2759),
 ('pm', 2724),
 ('narendra', 1566),
 ('rich', 1509),
 ('find', 1422)]

In [ ]:
raw_text=" ".join(dataset['text'])

mentions=[w for w in raw_text.split() if w.startswith("@") ]
print(Counter(mentions).most_common(7))

hashtags=[w for w in raw_text.split() if w.startswith("#") ]
hashtags=[w for w in hashtags if "demo" not in w.lower() ] #removes demonitization from list
print(Counter(hashtags).most_common(7))

links=[w for w in raw_text.split() if w.startswith("http") ]
print(Counter(links).most_common(7))

[('@evanspiegel', 1306), ('@URautelaForever:', 1273), ('@narendramodi', 1109), ('@gauravcsawant:', 541), ('@ModiBharosa:', 539), ('@DrKumarVishwas:', 350), ('@5Forty3:', 285)]
[('#India', 295), ('#nitishkumar', 257), ('#GLvMI', 145), ('#CorruptionFreeIndia', 103), ('#BlackMoney', 91), ('#VijayMallya', 86), ('#IndiaFightsCorruption', 79)]
[('https://t.co/ObQrhlNSL6', 350), ('https://t.co/UodwXdPMmG', 254), ('https://t', 252), ('https://t.c', 187), ('https:/', 162), ('https://t.co/DGAI5cf05y', 120), ('https://t.co/pYgK8Rmg7r', 114)]


In [ ]:
from nltk import ngrams
bigrams=ngrams(complete_text.split(),2)
print(Counter(bigrams).most_common(7))

[(('narendra', 'modi'), 1565), (('india', 'rich'), 1431), (('pm', 'narendra'), 1424), (('demonetization', 'find'), 1399), (('implement', 'demonetization'), 1397), (('rich', 'pm'), 1391), (('modi', 'implement'), 1377)]


In [ ]:

for text in dataset['text']:
  entities=nltk.ne_chunk(nltk.pos_tag(word_tokenize(text)))
  for entity in entities:
    if(hasattr(entity,"label")):
      print(entity)

(ORGANIZATION PM/NNP)
(GPE Modi/NNP)
(ORGANIZATION RBI/NNP Dy/NNP)
(ORGANIZATION CBDT/NNP)
(PERSON Harvard/NNP Professor/NNP)
(PERSON Aam/NNP Aadmi/NNP)
(ORGANIZATION Haryana/NNP)
(PERSON Reddy/NNP Wedding/NNP)
(GPE Kishtwar/NNP)
(ORGANIZATION CorruptionFreeIndia/NNP)
(GPE National/NNP)
(GPE Narendra/NNP)
(ORGANIZATION DeMonetization/NNP)
(ORGANIZATION Jaggesh2/NNP Bharat/NNP)
(ORGANIZATION Modi/NNP)
(ORGANIZATION DeMonetization/NNP)
(ORGANIZATION RBI/NNP Dy/NNP)
(ORGANIZATION CBDT/NNP)
(PERSON Harvard/NNP Professor/NNP)
(PERSON Aam/NNP Aadmi/NNP)
(ORGANIZATION AFTER/NNP Gandhi/NNP)
(GPE Kerala/NNP)
(PERSON Kerala/NNP)
(ORGANIZATION Black/NNP Money/NNP)
(ORGANIZATION RBI/NNP Dy/NNP)
(ORGANIZATION CBDT/NNP)
(PERSON Harvard/NNP Professor/NNP)
(PERSON Aam/NNP Aadmi/NNP)
(GPE Modi/NNP)
(ORGANIZATION RBI/NNP Dy/NNP)
(ORGANIZATION CBDT/NNP)
(PERSON Harvard/NNP Professor/NNP)
(PERSON Aam/NNP Aadmi/NNP)
(ORGANIZATION AFTER/NNP Gandhi/NNP)
(GPE Modi/NNP)
(ORGANIZATION RBI/NNP Dy/NNP)
(ORGANIZAT

KeyboardInterrupt: ignored

In [ ]:
# for text in dataset['text']:
#   entities=sent_tokenize(text)
#   for sentence in entities:
#     TextBlob(sentence).sentiment

In [ ]:
sentence="honest couldnt even use word molestation talking milo wo instant demonetization yet nword slips"
TextBlob(sentence).sentiment

Sentiment(polarity=0.3, subjectivity=0.7833333333333333)

In [ ]:
## Topic Modelling 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 

def generate_topic_models(text):
    cvectorizer = CountVectorizer(min_df=4, max_features=2000)
    cvz = cvectorizer.fit_transform(text)

    lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=20, random_state=42)
    X_topics = lda_model.fit_transform(cvz)

    topic_word = lda_model.components_ 
    vocab = cvectorizer.get_feature_names()
    return topic_word, vocab 

n_top_words = 10
topic_word, vocab = generate_topic_models(dataset["cleaned"].values)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print ("Topic " + str(i+1) + ": " + " | ".join(topic_words) + "\n")

Topic 1: demonetization | shortage | man | amp | ahead | govt | life | wedding | fund | gujarat

Topic 2: demonetization | whether | atms | still | today | got | videos | new | blackmoney | staff

Topic 3: demonetization | pm | narendra | dear | amp | modi | question | paytm | full | clearly

Topic 4: bank | demonetization | rs | terrorists | jampk | lakh | since | thats | third | 40

Topic 5: demonetization | youtube | good | bypolls | says | first | bjp | parliament | adityanath | ads

Topic 6: find | demonetization | india | money | black | running | digital | lost | claims | demo

Topic 7: demonetization | cash | modi | effect | result | note | mallyas | system | ban | quiet

Topic 8: india | demonetization | rich | modi | implement | actually | impact | poor | modis | yogi

Topic 9: demonetization | due | goes | india | oscar | mru092du093eu0935u0941u0915 | edu00a0u00bdedu00b8u00a9edu00a0u00bdedu00b8u00a5edu00a0u00bdedu00b8u00a2edu00a0u00bdedu00b8u00addemonetization | httpstcoobqr